# VPN Login from Non-Compliant Device

**VPN authentication success from device failing posture/compliance checks.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **device compliance reason**, **new device**, **impossible travel**, **concurrent logins**.


## Investigation Queries

Identify non-compliant VPN logins and verify device posture at the time.


In [ ]:
// KQL
VpnAuth
| where TimeGenerated > ago(30d)
| where User == '<USER>'
| where Result == 'Success'
| where DeviceCompliant == false
| project TimeGenerated, User, IPAddress, DeviceId, DeviceCompliant


In [ ]:
# Splunk SPL
index=vpn user=<USER> action=vpn_login outcome=success (device_compliant=false OR device_posture=noncompliant)
| table _time user src_ip device_id device_posture


In [ ]:
# Elastic (KQL/EQL)
event.category:vpn AND user.name:"<USER>" AND device.compliant:false


## Containment & Response

Terminate session, enforce conditional access/device compliance, and rotate credentials if needed.


In [ ]:
# PowerShell
## Revoke tokens / force re-auth
Revoke-AzureADUserAllRefreshToken -ObjectId <USER_OBJECT_ID>


In [ ]:
# PowerShell
## Document VPN control-plane action: terminate session / quarantine device (vendor-specific).
